In [ ]:
import caffe
import lmdb
from PIL import Image
import cv2
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os, time
import datetime
import random

video_files = os.listdir("videoclips")
annotation_files = os.listdir("annotations")

video_files.remove("example.avi")

def log(string):
    print "%s: %s" % (str(datetime.datetime.now())[:-6], string)
    
proportion = 0.25

with open("annotations/train_clean.txt") as fp:
    train_clean = fp.readlines()
    train_clean = [f[:-2] for f in train_clean]
    print len(train_clean)
    train_clean = np.random.choice(train_clean, int(len(train_clean) * proportion))

with open("annotations/test_clean.txt") as fp:
    test_clean = fp.readlines()
    test_clean = [f[:-2] for f in test_clean]
    test_clean = np.random.choice(test_clean, int(len(test_clean) * proportion))
    
with open("annotations/train_auto.txt") as fp:
    train_auto = fp.readlines()
    train_auto = [f[:-2] for f in train_auto]
    train_auto = np.random.choice(train_auto, int(len(train_auto) * proportion))

height = width = 100
    
def capture_and_release(fname, resize=(100, 100), trim=0.2, sobel=False, show=False):
    cap = cv2.VideoCapture(fname)
    rets, frames = [], []
    while True:
        ret, frame = cap.read()
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        
        if not ret:
            break
        
        rets.append(ret)
        if resize is not None:
            frame = np.array(Image.fromarray(np.array(frame).astype(np.uint8)).resize(resize))
        if sobel:
            frame = convolution(frame, kernels["sobel"])
            frame = frame.astype(np.float32)
        frame = frame / 255.
        frames.append(frame)


    if show:
        f1 = frames[1]
        cv_rgb = f1
        log("Shape: ", cv_rgb.shape)
        log("Number of frames: ", len(frames))
        if sobel:
            plt.imshow(cv_rgb, cmap="Greys_r")
        else:
            plt.imshow(cv_rgb)
        plt.show()

    cap.release()
    cv2.destroyAllWindows()

    if trim is not None:
        # Trim should < 0.5
        trim = int(len(frames) * trim)
        frames = frames[trim:-trim]
        log("Trimmed to %i frames" % len(frames))
    
    return frames

# Dictionary that has two levels:
# video file name => frame number => label (can have multiple)
video_to_frame_to_label = {fname: {} for fname in video_files}
stores = [train_clean, test_clean, train_auto]

label_bank = {"None": 0, "SitUp": 1, "GetOutCar": 2, "StandUp": 3, "AnswerPhone": 4,
             "Kiss": 5, "HugPerson": 6, "HandShake": 7, "SitDown": 8}

for idx, store in enumerate(stores):
    for line in store:
        video_title = line.split("\"")[1]
        begin, end = map(int, line.split("(")[1].split(")")[0].split("-"))
        labels = []
        label_group = line.split("<")[1:]
        for l in label_group:
            label = l.split(">")[0]
            labels.append(label_bank[label])
        for i in range(begin - 1, end + 1):
            if i in video_to_frame_to_label[video_title]:
                video_to_frame_to_label[video_title][i] = \
                        video_to_frame_to_label[video_title][i] + labels                           
            else:
                video_to_frame_to_label[video_title][i] = labels

def get_frames_and_labels(f, trim=0.4, sobel=False):
    """
    Assuming that f is of the form videoclips/FILENAME.avi
    """
    frames = capture_and_release(f, sobel=sobel)
    video_title = f.split("/")[1]
    return_frames = []
    return_labels = []
    for idx, frame in enumerate(frames):
        if frame is not None:
            if idx in video_to_frame_to_label[video_title]:
                labels = video_to_frame_to_label[video_title][idx]
    #             print "Labels length: ", len(labels)
                for label in labels:
                    return_frames.append(frame) # If a frame has two labels, we add two frames
                    return_labels.append(label)
    #                 print "Return frames length: ", len(return_frames)
    #                 print "Return labels length: ", len(labels)
            else:
                return_frames.append(frame)
                return_labels.append(0) # For non-annotated frames
    return return_frames, return_labels

trim = None

all_frames, all_labels = [], []
for f in train_clean:
    video = f.split("\"")[1]
    frames, labels = get_frames_and_labels("videoclips/" + video, trim=trim, sobel=False)
    
    if len(frames[0].shape) == 2:
        # Convert to 3-d data
        frames = np.array([frame[:,:,np.newaxis] for frame in frames])
    all_frames = all_frames + frames
    all_labels = all_labels + labels

In [ ]:
all_frames = np.array(all_frames)
print all_frames.shape

In [ ]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(all_frames, 
                                                        all_labels, test_size=0.1)

In [ ]:
import tensorflow as tf
import math

def autoencoder(input_shape=[None, height, width, 3],
                n_filters=[1, 3, 3, 3, 3, 3],
                filter_sizes=[3, 3, 3, 3, 3],
                corruption=False):
    # %%
    # input to the network
    x = tf.placeholder(tf.float32, input_shape, name='x')


    # %%
    # ensure 2-d is converted to square tensor.
    if len(x.get_shape()) == 2:
        x_dim = np.sqrt(x.get_shape().as_list()[1])
        if x_dim != int(x_dim):
            raise ValueError('Unsupported input dimensions')
        x_dim = int(x_dim)
        x_tensor = tf.reshape(
            x, [-1, x_dim, x_dim, n_filters[0]])
    elif len(x.get_shape()) == 4:
        x_tensor = x
    else:
        raise ValueError('Unsupported input dimensions')
    current_input = x_tensor

    # %%
    # Build the encoder
    encoder_weights = []
    encoder_ops = []
    shapes = []
    for layer_i, n_output in enumerate(n_filters[1:]):
        n_input = current_input.get_shape().as_list()[3]
        shapes.append(current_input.get_shape().as_list())
        W = tf.Variable(
            tf.random_uniform([
                filter_sizes[layer_i],
                filter_sizes[layer_i],
                n_input, n_output],
                -1.0 / math.sqrt(n_input),
                1.0 / math.sqrt(n_input)))
        b = tf.Variable(tf.zeros([n_output]))
        encoder_weights.append(W)
        output = tf.nn.sigmoid(
            tf.add(tf.nn.conv2d(
                current_input, W, strides=[1, 1, 1, 1], padding='SAME'), b))
        encoder_ops.append(output)
        current_input = output

    # %%
    # store the latent representation
    z = current_input
    encoder_weights.reverse()
    shapes.reverse()

    # %%
    # Build the decoder using the same weights
    for layer_i, shape in enumerate(shapes):
        W = encoder_weights[layer_i]
        b = tf.Variable(tf.zeros([W.get_shape().as_list()[2]]))
        output = tf.nn.sigmoid(
            tf.add(tf.nn.conv2d_transpose(
                current_input, W,
                tf.pack([tf.shape(x)[0], shape[1], shape[2], shape[3]]),
                strides=[1, 1, 1, 1], padding='SAME'), b))
        current_input = output
        
    decoder = current_input

    # %%
    # now have the reconstruction through the network
    y = current_input
    # cost function measures pixel-wise difference
    cost = tf.reduce_sum(tf.square(y - x_tensor))

    # %%
    return {'x': x, 'z': z, 'y': y, 'cost': cost, 
            "encoder": encoder_ops, "decoder": decoder}


# %%
def test_hollywood(X_train, X_test, n_filters, filter_sizes):
    import tensorflow as tf
    ae = autoencoder(n_filters=n_filters, filter_sizes=filter_sizes)

    # %%
    learning_rate = 0.001
    optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(ae['cost'])

    # %%
    # We create a session to use the graph
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    sess.run(tf.initialize_all_variables())

    # %%
    # Fit all training data
    batch_size = 100
    n_epochs = 100
    step_size = 10
    for epoch_i in range(n_epochs):
        for batch_i in range(X_train.shape[0] // batch_size):
            batch_xs = X_train[batch_i * batch_size:(batch_i + 1) * batch_size]
            train = batch_xs
            sess.run(optimizer, feed_dict={ae['x']: train})
        if epoch_i % step_size == 0:
            print(str(datetime.datetime.now()), epoch_i, sess.run(ae['cost'], feed_dict={ae['x']: train}))

    # %%
    # Plot example reconstructions
    n_examples = 5
    test_xs = X_test[:n_examples]
    test_xs_norm = test_xs
    recon = sess.run(ae['y'], feed_dict={ae['x']: test_xs_norm})
    print(recon.shape)
    fig, axs = plt.subplots(2, n_examples, figsize=(30, 15))
    for example_i in range(n_examples):
        axs[0][example_i].imshow(
            test_xs[example_i, :].reshape((height, width, 3))[:,:,::-1])
        axs[0][example_i].axis("off")

        axs[1][example_i].imshow(
            recon[example_i, ...].reshape((height, width, 3))[:,:,::-1])
#                 np.reshape(recon[example_i, ...], (height, width, 3)))
        axs[1][example_i].axis("off")
    
    fig.show()
    plt.draw()
    
    ae["session"] = sess
    
    return ae

In [ ]:
ae = test_hollywood(X_train, X_test, n_filters=[3, 5, 5, 5, 5],
                filter_sizes=[3, 3, 3, 3])

In [ ]:
del all_frames

In [ ]:
# train = X_train[:2000]
# sess = ae["session"]
# layers = [sess.run(ae["encoder"][i], 
#         feed_dict={ae['x']: train}) for i in range(len(ae["encoder"]))]
# ravels = (np.array([row.ravel() for row in layers[i]]) for i in range(len(ae["encoder"])))
# combined = np.hstack(ravels)
# print combined.shape

train = X_train
combined = []
sess = ae["session"]
batch_size = 100
for batch_i in range(train.shape[0] // batch_size):
    batch_xs = train[batch_i * batch_size:(batch_i + 1) * batch_size]
    layers = [sess.run(ae["encoder"][i], 
            feed_dict={ae['x']: batch_xs}) for i in range(len(ae["encoder"]))]
    ravels = (np.array([row.ravel() for row in layers[i]]) for i in range(len(ae["encoder"])))
    interm = np.hstack((ravels))
    combined.append(interm)
    
combined = np.vstack((combined))
print combined.shape

In [ ]:
# # Scale and visualize the embedding vectors
# def plot_embedding(X, y, title=None):
#     x_min, x_max = np.min(X, 0), np.max(X, 0)
#     X = (X - x_min) / (x_max - x_min)

#     plt.figure(figsize=(20, 10))
#     ax = plt.subplot(111)
#     for i in range(X.shape[0]):
#         plt.text(X[i, 0], X[i, 1], str(y[i]),
#                  color=plt.cm.Set1(y[i] / 10.),
#                  fontdict={'weight': 'bold', 'size': 12})

#     plt.xticks([]), plt.yticks([])
#     if title is not None:
#         plt.title(title)

# vectorized_imgs = combined

# from sklearn.manifold import TSNE

# tsne = TSNE(n_components=2, random_state=0)
# np.set_printoptions(suppress=True)
# X_tsne = tsne.fit_transform(vectorized_imgs)

In [ ]:
# plot_embedding(X_tsne, y_train[:X_tsne.shape[0]])

In [ ]:
# from sklearn.neigkhbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors=3)
# knn.fit(X_tsne, y_train[:X_tsne.shape[0]])

In [ ]:
# knn.score(X_tsne, y_train[:X_tsne.shape[0]])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(combined, y_train[:combined.shape[0]])
print str(datetime.datetime.now())

In [ ]:
knn.score(combined, y_train[:combined.shape[0]])
print str(datetime.datetime.now())

In [ ]:
y_pred = knn.predict(combined)
print str(datetime.datetime.now())
y = y_train[:combined.shape[0]]

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import numpy as np

plt.figure(figsize=(30, 30))

class_names = ["None", "AnswerPhone", "GetOutCar", "HandShake", "HugPerson", "Kiss", "SitDown", "SitUp", "StandUp"]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
#     plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
#     plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, str(cm[i, j])[:4],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

cnf_matrix = confusion_matrix(y_train[:combined.shape[0]][:1000], y_pred[:1000])
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix for HoHA')

plt.show()

# Nearest Neighbor Querying

In [ ]:
y_train = np.array(y_train)
np.where(y_train == 0)

In [ ]:
from scipy.spatial import cKDTree
import random

# test_idxes = random.sample(range(combined.shape[0]), 10)
test_idxes = random.sample(np.where(y_train == 0)[0], 10)
print test_idxes

tree = cKDTree(combined)
test_array = combined[test_idxes]
query_res = tree.query(test_array, k=5)

nns = []

for idx, row in enumerate(query_res[1]):
    nn = X_train[row.ravel()]
#     to_plot = np.vstack([X_test[idx], nn])
    nns.append(nn)
    
nns = np.stack(nns)

In [ ]:
nns.shape

In [ ]:
def plot_nearest_neighbors(nns, reshape=(100, 100, 3)):
    fig, axs = plt.subplots(nns.shape[0], nns.shape[1], figsize=(32, 32))
    
    for i in range(nns.shape[0]):
        for j in range(nns.shape[1]):
            if len(reshape) != 2:
                axs[i][j].imshow(
                    nns[i, j].reshape(reshape)[:,:,::-1])
            else:
                axs[i][j].imshow(
                    nns[i, j].reshape(reshape), cmap="Greys_r")
            axs[i][j].axis("off")
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.tight_layout()

In [ ]:
plot_nearest_neighbors(nns)

# Just k-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
X_train_ravelled = np.array([row.ravel() for row in X_train])
knn.fit(X_train_ravelled, y_train)

In [ ]:
knn.score(X_train_ravelled, y_train)

In [ ]:
X_train_ravelled = np.array([row.ravel() for row in X_train])
X_train = X_train_ravelled[:5000]
y_train = y_train[:5000]

# Just k-means

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=9, random_state=0)
kmeans.fit(X_train)

def cluster_acc(Y_pred, Y):
    """
    Finds the cluster accuracy
    """
    from sklearn.utils.linear_assignment_ import linear_assignment
    Y_pred = np.array(Y_pred)
    Y = np.array(Y)
    D = max(Y_pred.max(), Y.max())+1
    w = np.zeros((D,D), dtype=np.int64)
    for i in xrange(Y_pred.size):
        w[Y_pred[i], Y[i]] += 1
    ind = linear_assignment(w.max() - w)
    return sum([w[i,j] for i,j in ind])*1.0/Y_pred.size, w

y_pred = kmeans.predict(X_train)
cluster_acc(y_pred, y_train)

# Just t-SNE

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
np.set_printoptions(suppress=True)
X_tsne = tsne.fit_transform(X_train)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_tsne, y_train[:X_tsne.shape[0]])

In [ ]:
knn.score(X_tsne, y_train[:X_tsne.shape[0]])